# PART- 2 `( Data Fill )`

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats.mstats import winsorize

import warnings
warnings.filterwarnings("ignore")
warnings.warn("this will not show")

%matplotlib inline
# %matplotlib notebook

plt.rcParams["figure.figsize"] = (10,6)
# plt.rcParams['figure.dpi'] = 100

sns.set_style("whitegrid")
pd.set_option('display.float_format', lambda x: '%.3f' % x)

pd.options.display.max_rows = 1000
pd.options.display.max_columns = 150

In [2]:
df_car = pd.read_csv("df_car_cleaned.csv")

In [3]:
df_car

,make_model,body_type,price,vat,km,car_age,prev_owner,horse_power,color,paint_type,gears,safety_security,country_version,type,warranty,previous_owner,upholstery,nr_of_doors,nr_of_seats,gearing_type,displacement,cylinders,weight_kg,drive_chain,fuel,consump_comb,consump_city,consump_country,co2_emission,comfort_convenience,media,extras
0,Audi A1,Sedans,15770,VAT deductible,56013.000,3.000,2.000,66.000,Black,Metallic,NaN,"abs,central door lock,daytime running lights,d...",NaN,Used,NaN,2.000,Cloth,5.000,5.000,Automatic,1400.000,3.000,1220.000,front,Diesel,3.800,4.300,3.500,99.000,"Air conditioning,Armrest,Automatic climate con...","Bluetooth,Hands-free equipment,On-board comput...","Alloy wheels,Catalytic Converter,Voice Control"
1,Audi A1,Sedans,14500,Price negotiable,80000.000,2.000,NaN,141.000,Red,NaN,7.000,"abs,central door lock,central door lock with r...",NaN,Used,NaN,NaN,Cloth,3.000,4.000,Automatic,1800.000,4.000,1255.000,front,Benzine,5.600,7.100,4.700,129.000,"Air conditioning,Automatic climate control,Hil...","Bluetooth,Hands-free equipment,On-board comput...","Alloy wheels,Sport seats,Sport suspension,Voic..."
2,Audi A1,Sedans,14640,VAT deductible,83450.000,3.000,1.000,85.000,Black,Metallic,NaN,"abs,central door lock,daytime running lights,d...",NaN,Used,NaN,1.000,Cloth,4.000,4.000,Automatic,1600.000,NaN,NaN,front,Diesel,3.800,4.400,3.400,99.000,"Air conditioning,Cruise control,Electrical sid...","MP3,On-board computer","Alloy wheels,Voice Control"
3,Audi A1,Sedans,14500,NaN,73000.000,3.000,1.000,66.000,Brown,Metallic,6.000,"abs,alarm system,central door lock with remote...",NaN,Used,NaN,1.000,NaN,3.000,4.000,Automatic,1400.000,3.000,1195.000,NaN,Diesel,3.800,4.300,3.500,99.000,"Air suspension,Armrest,Auxiliary heating,Elect...","Bluetooth,CD player,Hands-free equipment,MP3,O...","Alloy wheels,Sport seats,Voice Control"
4,Audi A1,Sedans,16790,NaN,16200.000,3.000,1.000,66.000,Black,Metallic,NaN,"abs,central door lock,driver-side airbag,elect...",Germany,Used,NaN,1.000,Cloth,5.000,5.000,Automatic,1400.000,3.000,NaN,front,Diesel,4.100,4.600,3.800,109.000,"Air conditioning,Armrest,Automatic climate con...","Bluetooth,CD player,Hands-free equipment,MP3,O...","Alloy wheels,Sport package,Sport suspension,Vo..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15914,Renault Espace,Van,39950,VAT deductible,NaN,NaN,NaN,147.000,Grey,Metallic,6.000,"abs,central door lock,central door lock with r...",NaN,New,24.000,NaN,NaN,5.000,5.000,Automatic,2000.000,4.000,1758.000,front,Diesel,5.300,6.200,4.700,139.000,"Air conditioning,Automatic climate control,Cru...","Bluetooth,Digital radio,Hands-free equipment,O...","Alloy wheels,Touch screen"
15915,Renault Espace,Van,39885,VAT deductible,9900.000,0.000,1.000,165.000,Grey,Metallic,7.000,"abs,adaptive cruise control,blind spot monitor...",Germany,Used,NaN,1.000,Cloth,5.000,5.000,Automatic,1800.000,4.000,1708.000,front,Benzine,7.400,9.200,6.300,168.000,"Air conditioning,Automatic climate control,Cru...","Bluetooth,Digital radio,Hands-free equipment,R...","Alloy wheels,Touch screen,Voice Control"
15916,Renault Espace,Van,39875,VAT deductible,15.000,0.000,1.000,146.000,White,NaN,6.000,"abs,adaptive cruise control,blind spot monitor...",Austria,Pre-registered,NaN,1.000,Full leather,5.000,7.000,Automatic,2000.000,4.000,NaN,front,Diesel,5.300,6.200,4.700,139.000,"Air conditioning,Armrest,Automatic climate con...","Bluetooth,Hands-free equipment,On-board comput...",Alloy wheels
15917,Renault Espace,Van,39700,VAT deductible,10.000,0.000,NaN,147.000,Grey,NaN,6.000,"abs,blind spot monitor,driver-side airbag,elec...",NaN,Pre-registered,NaN,NaN,Part leather,5.000,7.000,Automatic,2000.000,4.000,1758.000,front,Diesel,5.300,6.200,4.700,139.000,"Air conditioning,Automatic climate control,Cru...","Bluetooth,Digital radio,Radio,USB","Alloy wheels,Touch screen"


In [4]:
def groupby_mode2(group_col1, group_col2, fill_col):
    for group1 in list(df_car[group_col1].unique()):
        cond1 = (df_car[group_col1]==group1)
        
        for group2 in list(df_car[group_col2].unique()):
            cond2 = (df_car[group_col2]==group2)
            cond = (cond1 & cond2)
            
            mode2 = list(df_car[cond][fill_col].mode())
            
            if mode2 != []:
                df_car.loc[cond,fill_col] = df_car.loc[cond,fill_col].fillna(mode2[0])
            elif list(df_car[cond1][fill_col].mode()) != []:
                df_car.loc[cond,fill_col] = df_car.loc[cond,fill_col].fillna(df_car[cond1][fill_col].mode()[0])
            else:
                df_car.loc[cond,fill_col] = df_car.loc[cond,fill_col].fillna(df_car[fill_col].mode()[0])
    print("Number of NaN : ",df_car[fill_col].isnull().sum())
    print("------------------")
    print(df_car[fill_col].value_counts(dropna=False))

def fill_most(df, group_col, col_name):
    '''Fills the missing values with the most existing value (mode) in the relevant column according to single-stage grouping'''
    for group in list(df[group_col].unique()):
        cond = df[group_col]==group
        mode = list(df[cond][col_name].mode())
        if mode != []:
            df.loc[cond, col_name] = df.loc[cond, col_name].fillna(df[cond][col_name].mode()[0])
        else:
            df.loc[cond, col_name] = df.loc[cond, col_name].fillna(df[col_name].mode()[0])
    print("Number of NaN : ",df[col_name].isnull().sum())
    print("------------------")
    print(df[col_name].value_counts(dropna=False))


## Let's examine and fill the missing values of all the columns/features one by one

## `make_model`

In [5]:
df_car.make_model.value_counts(dropna=False)

Audi A3           3097
Audi A1           2614
Opel Insignia     2598
Opel Astra        2526
Opel Corsa        2219
Renault Clio      1839
Renault Espace     991
Renault Duster      34
Audi A2              1
Name: make_model, dtype: int64

## `body_type`

In [6]:
df_car.body_type.value_counts(dropna=False)

Sedans           7903
Station wagon    3553
Compact          3153
Van               783
Other             290
Transporter        88
NaN                60
Off-Road           56
Coupe              25
Convertible         8
Name: body_type, dtype: int64

In [7]:
df_car.groupby("make_model").body_type.value_counts(dropna=False)

make_model      body_type    
Audi A1         Sedans           1538
                Compact          1039
                Station wagon      21
                Other              13
                Coupe               2
                Van                 1
Audi A2         Off-Road            1
Audi A3         Sedans           2598
                Station wagon     282
                Compact           182
                Other              16
                Convertible         8
                NaN                 7
                Coupe               4
Opel Astra      Station wagon    1211
                Sedans           1053
                Compact           185
                Other              67
                NaN                 7
                Coupe               2
                Off-Road            1
Opel Corsa      Compact          1230
                Sedans            875
                Other              87
                Coupe              13
                Tran

In [8]:
fill_most(df_car, 'make_model',"body_type")

Number of NaN :  0
------------------
Sedans           7925
Station wagon    3563
Compact          3155
Van               809
Other             290
Transporter        88
Off-Road           56
Coupe              25
Convertible         8
Name: body_type, dtype: int64


## `price`

In [9]:
df_car.price.isnull().sum()

0

## `color`

In [10]:
df_car.color.value_counts(dropna=False)

Black     3745
Grey      3505
White     3406
Silver    1647
Blue      1431
Red        957
NaN        597
Brown      289
Green      154
Beige      108
Yellow      51
Violet      18
Bronze       6
Orange       3
Gold         2
Name: color, dtype: int64

In [11]:
df_car.groupby(["make_model","car_age"])["color"].value_counts(dropna=False)

make_model      car_age  color 
Audi A1         0.000    White     169
                         Black      86
                         Grey       71
                         Blue       65
                         Red        45
                         Green      18
                         Yellow     13
                         NaN        12
                         Silver      4
                1.000    Black     209
                         White     169
                         Grey      100
                         Red        90
                         Blue       88
                         Silver     60
                         NaN        14
                         Beige      10
                         Green       3
                         Yellow      1
                2.000    White     112
                         Black     109
                         Blue       60
                         Grey       47
                         Red        44
                         Silver 

In [12]:
groupby_mode2("make_model","car_age","color")

Number of NaN :  420
------------------
Black     3800
Grey      3588
White     3442
Silver    1650
Blue      1431
Red        957
NaN        420
Brown      289
Green      154
Beige      108
Yellow      51
Violet      18
Bronze       6
Orange       3
Gold         2
Name: color, dtype: int64


In [13]:
fill_most(df_car,"make_model","color")

Number of NaN :  0
------------------
Grey      3904
Black     3844
White     3502
Silver    1650
Blue      1431
Red        957
Brown      289
Green      154
Beige      108
Yellow      51
Violet      18
Bronze       6
Orange       3
Gold         2
Name: color, dtype: int64


## `Paint Type`

In [14]:
df_car.groupby(["make_model","color"])["paint_type"].value_counts(dropna=False)

make_model      color   paint_type
Audi A1         Beige   Metallic       15
                        NaN             1
                        Uni             1
                Black   Metallic      515
                        NaN           211
                        Uni             8
                Blue    Metallic      236
                        NaN            45
                        Uni             5
                Brown   Metallic       20
                        NaN             2
                Green   NaN            24
                        Metallic        7
                        Uni             1
                Grey    Metallic      272
                        NaN            49
                        Uni             6
                Red     Metallic      191
                        NaN            46
                        Uni            12
                        Perl            1
                Silver  Metallic      113
                        NaN            13

In [15]:
groupby_mode2("make_model","color","paint_type")

Number of NaN :  0
------------------
Metallic    15547
Uni           364
Perl            8
Name: paint_type, dtype: int64


In [16]:
fill_most(df_car,"make_model","paint_type")

Number of NaN :  0
------------------
Metallic    15547
Uni           364
Perl            8
Name: paint_type, dtype: int64


## `Gears`

In [17]:
df_car.groupby(["make_model","gearing_type"])["gears"].value_counts(dropna=False)

make_model      gearing_type    gears
Audi A1         Automatic       7.000     600
                                NaN       515
                                6.000      18
                                5.000      17
                                8.000       2
                Manual          5.000     801
                                NaN       436
                                6.000     167
                                7.000       1
                Semi-automatic  7.000      55
                                NaN         1
                                6.000       1
Audi A2         Manual          NaN         1
Audi A3         Automatic       7.000     893
                                NaN       385
                                6.000      82
                Manual          6.000    1127
                                NaN       340
                                7.000       8
                                5.000       2
                Semi-automatic  7.000     

In [18]:
groupby_mode2("make_model","gearing_type","gears")

Number of NaN :  0
------------------
6.000    8617
5.000    4255
7.000    2810
8.000     224
9.000       6
4.000       2
1.000       2
3.000       2
2.000       1
Name: gears, dtype: int64


## `country_version`

In [19]:
df_car['country_version'].value_counts(dropna=False)

NaN            8333
Germany        4502
Italy          1038
European        507
Netherlands     464
Spain           325
Belgium         314
Austria         208
Czech            52
Poland           49
France           38
Denmark          33
Hungary          28
Japan             8
Slovakia          4
Croatia           4
Sweden            3
Bulgaria          2
Romania           2
Serbia            1
Egypt             1
Luxembourg        1
Switzerland       1
Slovenia          1
Name: country_version, dtype: int64

Too many NaN, and no clue for filling, so drop column

In [20]:
df_car.drop('warranty',axis=1,inplace=True)

## `vat`

In [21]:
df_car['vat'].value_counts(dropna=False)

VAT deductible      10980
NaN                  4513
Price negotiable      426
Name: vat, dtype: int64

Too many NaNs and mostly VAT deductable, so no info can be obtained

In [22]:
df_car.drop('vat',axis=1,inplace=True)

## `prev_owner`

In [23]:
df_car.drop(columns=['previous_owner'],axis=1,inplace=True)

In [24]:
df_car['prev_owner'].value_counts(dropna=False)

1.000    8294
NaN      6828
2.000     778
3.000      17
4.000       2
Name: prev_owner, dtype: int64

In [25]:
df_car.groupby('type').prev_owner.value_counts(dropna=False)

type            prev_owner
Demonstration   1.000          508
                NaN            282
                2.000            6
Employee's car  1.000          641
                NaN            360
                2.000           10
New             NaN           1635
                1.000           15
Pre-registered  1.000          839
                NaN            519
                2.000            6
Used            1.000         6291
                NaN           4030
                2.000          756
                3.000           17
                4.000            2
Name: prev_owner, dtype: int64

In [26]:
cond = (df_car.type=="New") | (df_car.type=="Pre-registered")
df_car.loc[cond,'prev_owner'] = df_car.loc[cond,'prev_owner'].fillna(value=0)

In [27]:
cond.sum()

3014

In [28]:
df_car['prev_owner'].value_counts(dropna=False)

1.000    8294
NaN      4674
0.000    2154
2.000     778
3.000      17
4.000       2
Name: prev_owner, dtype: int64

In [29]:
df_car['prev_owner'].fillna(1.0,inplace=True)

In [30]:
df_car['prev_owner'].value_counts(dropna=False)

1.000    12968
0.000     2154
2.000      778
3.000       17
4.000        2
Name: prev_owner, dtype: int64

## `displacement`

In [31]:
groupby_mode2("make_model","gearing_type","displacement")

Number of NaN :  0
------------------
1600.000     5384
1400.000     3109
1000.000     2587
1500.000     1476
1200.000     1246
2000.000     1164
900.000       372
1100.000      289
1800.000      211
0.000          38
2500.000       20
100.000         7
200.000         6
1300.000        3
1900.000        2
1700.000        2
15900.000       1
3000.000        1
16000.000       1
Name: displacement, dtype: int64


## `horse_power`

In [32]:
df_car['horse_power'].isnull().sum()

88

In [33]:
groupby_mode2('make_model','displacement','horse_power')

Number of NaN :  0
------------------
85.000     2543
66.000     2123
81.000     1402
100.000    1318
110.000    1114
70.000      890
125.000     707
51.000      696
55.000      588
118.000     550
92.000      466
121.000     392
147.000     380
77.000      353
56.000      286
54.000      276
103.000     253
87.000      242
165.000     194
88.000      177
60.000      160
162.000      98
74.000       81
96.000       72
71.000       59
101.000      47
67.000       40
154.000      39
122.000      35
119.000      30
164.000      27
135.000      24
52.000       22
82.000       22
78.000       20
1.000        20
146.000      18
294.000      18
141.000      16
57.000       10
120.000       8
104.000       8
191.000       7
112.000       7
155.000       6
117.000       6
184.000       5
76.000        4
65.000        4
90.000        4
80.000        3
98.000        3
168.000       3
93.000        3
149.000       3
150.000       2
89.000        2
63.000        2
86.000        2
53.000        2
27

In [34]:
(df_car.horse_power < 60).sum()

1905

## `country_version`

In [35]:
df_car.drop(columns=['country_version'],axis=1,inplace=True)

## `type`

In [36]:
df_car.type.isnull().sum()

2

In [37]:
df_car.type.fillna("Used",inplace=True)

## `car_age`

In [38]:
df_car.car_age.isnull().sum()

1597

In [39]:
df_car.groupby("type").car_age.value_counts(dropna=False)

type            car_age
Demonstration   0.000       632
                1.000       130
                2.000        27
                NaN           5
                3.000         2
Employee's car  1.000       765
                0.000       127
                2.000        99
                3.000        17
                NaN           3
New             NaN        1547
                0.000       100
                1.000         3
Pre-registered  0.000      1187
                1.000       161
                2.000         9
                NaN           6
                3.000         1
Used            3.000      3654
                1.000      3463
                2.000      3138
                0.000       807
                NaN          36
Name: car_age, dtype: int64

In [40]:
cond = (df_car.type=="New") | (df_car.type=="Pre-registered")

In [41]:
df_car.loc[cond,"car_age"] = 0

In [42]:
fill_most(df_car,"type","car_age")

Number of NaN :  0
------------------
0.000    4585
1.000    4361
3.000    3709
2.000    3264
Name: car_age, dtype: int64


## `km`

In [43]:
for age in list(df_car["car_age"].unique()):
    cond = (df_car["car_age"]==age)
    mean_km = df_car.loc[cond, "km"].mean()

    df_car.loc[cond,"km"] = df_car.loc[cond,"km"].fillna(mean_km)
    

In [44]:
df_car.km.isnull().sum()

0

## `safety_security`

In [45]:
df_car.safety_security.isnull().sum()

982

In [46]:
groupby_mode2("make_model","body_type","safety_security")

Number of NaN :  0
------------------
abs,central door lock,daytime running lights,driver-side airbag,electronic stability control,fog lights,immobilizer,isofix,passenger-side airbag,power steering,side airbag,tire pressure monitoring system,traction control                                          729
abs,central door lock,daytime running lights,driver-side airbag,electronic stability control,immobilizer,isofix,passenger-side airbag,power steering,side airbag,tire pressure monitoring system,traction control                                                     480
abs,central door lock,daytime running lights,driver-side airbag,electronic stability control,fog lights,immobilizer,isofix,led daytime running lights,passenger-side airbag,power steering,side airbag,tire pressure monitoring system,traction control               373
abs,central door lock,daytime running lights,driver-side airbag,electronic stability control,immobilizer,isofix,passenger-side airbag,power steering,side airbag,tir

## `upholstery`

In [47]:
df_car.groupby(["make_model","body_type"])["upholstery"].value_counts(dropna=False)


make_model      body_type      upholstery  
Audi A1         Compact        Cloth            807
                               NaN               89
                               Part leather      48
                               Other             34
                               Black             27
                               Velour            15
                               alcantara          7
                               Full leather       6
                               Grey               5
                               Blue               1
                Coupe          NaN                1
                               Part leather       1
                Other          Cloth             12
                               NaN                1
                Sedans         Cloth            862
                               NaN              504
                               Part leather      67
                               Other             53
                    

In [48]:
groupby_mode2("make_model","body_type","upholstery")

Number of NaN :  0
------------------
Cloth           11646
Part leather     1549
Full leather     1333
Black             614
Other             368
Grey              273
Velour             60
alcantara          57
Brown              12
Beige               3
White               2
Blue                2
Name: upholstery, dtype: int64


## `nr_of_doors`

In [49]:
df_car.groupby(["make_model","body_type"])["nr_of_doors"].value_counts(dropna=False)


make_model      body_type      nr_of_doors
Audi A1         Compact        5.000           666
                               4.000           207
                               3.000            80
                               2.000            69
                               NaN              17
                Coupe          2.000             1
                               5.000             1
                Other          5.000             8
                               4.000             5
                Sedans         5.000          1048
                               4.000           321
                               3.000           130
                               2.000            29
                               NaN              10
                Station wagon  5.000            17
                               4.000             3
                               3.000             1
                Van            5.000             1
Audi A2         Off-Road       5.000   

In [50]:
groupby_mode2("make_model","body_type","nr_of_doors")

Number of NaN :  0
------------------
5.000    11787
4.000     3079
3.000      832
2.000      219
7.000        1
1.000        1
Name: nr_of_doors, dtype: int64


## `nr_of_seats`

In [51]:
df_car.groupby(["make_model","body_type"])["nr_of_seats"].value_counts(dropna=False)


make_model      body_type      nr_of_seats
Audi A1         Compact        5.000           538
                               4.000           431
                               NaN              68
                               3.000             1
                               6.000             1
                Coupe          4.000             1
                               5.000             1
                Other          5.000            12
                               4.000             1
                Sedans         5.000           827
                               4.000           641
                               NaN              70
                Station wagon  5.000            14
                               4.000             6
                               NaN               1
                Van            NaN               1
Audi A2         Off-Road       5.000             1
Audi A3         Compact        5.000           175
                               NaN     

In [52]:
groupby_mode2("make_model","body_type","nr_of_seats")

Number of NaN :  0
------------------
5.000    14308
4.000     1127
7.000      362
2.000      119
6.000        2
3.000        1
Name: nr_of_seats, dtype: int64


## `gearing_type`

In [53]:
df_car.gearing_type.isnull().sum()

0

In [54]:
df_car.gearing_type.value_counts(dropna=True)

Manual            8153
Automatic         7297
Semi-automatic     469
Name: gearing_type, dtype: int64

## `cylinders`

In [55]:
df_car.cylinders.isnull().sum()

5680

In [56]:
groupby_mode2("make_model","displacement","cylinders")

Number of NaN :  0
------------------
4.000    12413
3.000     3476
5.000       22
6.000        3
8.000        2
2.000        2
1.000        1
Name: cylinders, dtype: int64


## `weight_kg`

In [57]:
df_car.weight_kg.isnull().sum()

6974

In [58]:
groupby_mode2("make_model","body_type","weight_kg")

Number of NaN :  0
------------------
1163.000    1582
1360.000    1406
1487.000     957
1135.000     828
1425.000     731
1180.000     694
1273.000     656
1165.000     589
1503.000     570
1734.000     553
1087.000     291
1335.000     242
1365.000     211
1199.000     205
1350.000     156
1119.000     153
1355.000     136
1260.000     127
1280.000     127
1278.000     123
1275.000     112
1255.000     108
1200.000     107
1522.000     103
1659.000     102
1195.000      96
1120.000      93
1403.000      91
1701.000      87
1250.000      85
1685.000      83
1441.000      82
1285.000      80
1308.000      80
1090.000      77
1613.000      75
1110.000      75
1279.000      72
1364.000      70
1345.000      67
1733.000      65
1325.000      64
1071.000      64
1209.000      64
1141.000      64
1230.000      63
1205.000      59
1845.000      56
1052.000      53
1154.000      52
1664.000      52
1513.000      51
1065.000      50
1237.000      49
1440.000      46
1088.000      46
1265.000  

## `drive_chain`

In [59]:
df_car.drive_chain.value_counts(dropna=False)

front    8886
NaN      6858
4WD       171
rear        4
Name: drive_chain, dtype: int64

In [60]:
groupby_mode2("make_model","body_type","drive_chain")

Number of NaN :  0
------------------
front    15711
4WD        204
rear         4
Name: drive_chain, dtype: int64


## `fuel`

In [61]:
df_car.fuel.value_counts(dropna=False)

Benzine     8551
Diesel      7299
LPG/CNG       64
Electric       5
Name: fuel, dtype: int64

## `co2_emission`

In [62]:
df_car.co2_emission.isnull().sum()

2436

In [63]:
groupby_mode2("make_model","displacement","co2_emission")

Number of NaN :  0
------------------
120.000    1122
97.000      784
99.000      737
114.000     677
119.000     652
102.000     531
124.000     512
104.000     505
107.000     472
85.000      471
103.000     445
128.000     442
108.000     362
106.000     349
117.000     330
150.000     301
127.000     296
126.000     282
113.000     274
118.000     270
110.000     269
139.000     238
111.000     237
109.000     234
141.000     231
140.000     218
129.000     215
135.000     204
105.000     202
92.000      198
130.000     180
123.000     178
112.000     171
143.000     171
95.000      161
116.000     157
98.000      151
136.000     148
133.000     145
153.000     139
137.000     133
125.000     132
134.000     130
145.000     126
149.000     117
147.000     109
101.000     105
132.000     100
115.000      91
122.000      83
121.000      82
138.000      75
93.000       66
142.000      58
168.000      58
148.000      48
131.000      48
144.000      40
154.000      40
94.000       37
10

## `comfort_convenience`

In [64]:
groupby_mode2("make_model","body_type","comfort_convenience")

Number of NaN :  0
------------------
Air conditioning,Electrical side mirrors,Hill Holder,Power windows                                                                                                                                                                                                                                                                                                                                                       315
Air conditioning,Armrest,Automatic climate control,Cruise control,Electrical side mirrors,Leather steering wheel,Light sensor,Lumbar support,Multi-function steering wheel,Navigation system,Park Distance Control,Parking assist system sensors front,Parking assist system sensors rear,Power windows,Rain sensor,Seat heating,Start-stop system                                                                       187
Air conditioning,Automatic climate control,Cruise control,Multi-function steering wheel,Park Distance Control,Power windows             

In [65]:
df_car.comfort_convenience.isnull().sum()

0

## `media`

In [66]:
groupby_mode2("make_model","body_type","media")

Number of NaN :  0
------------------
Bluetooth,Hands-free equipment,On-board computer,Radio,USB                                                        1639
Bluetooth,Hands-free equipment,MP3,On-board computer,Radio,USB                                                    1063
Bluetooth,CD player,Hands-free equipment,MP3,On-board computer,Radio,USB                                          1007
On-board computer                                                                                                  681
Radio                                                                                                              558
Bluetooth,Hands-free equipment,On-board computer,Radio                                                             514
On-board computer,Radio                                                                                            487
Bluetooth,CD player,Hands-free equipment,On-board computer,Radio,USB                                               466
Bluetooth,

In [67]:
df_car.media.isnull().sum()

0

## `extras`

In [68]:
groupby_mode2("make_model","body_type","extras")

Number of NaN :  0
------------------
Alloy wheels                                                                                                                                   5884
Alloy wheels,Touch screen                                                                                                                       697
Alloy wheels,Voice Control                                                                                                                      582
Alloy wheels,Touch screen,Voice Control                                                                                                         544
Roof rack                                                                                                                                       538
Alloy wheels,Roof rack                                                                                                                          487
Alloy wheels,Sport seats                                                  

In [69]:
df_car.extras.isnull().sum()

0

## `consump_city`

In [70]:
df_car.consump_city.isnull().sum()

4039

In [71]:
groupby_mode2("make_model","displacement","consump_city")

Number of NaN :  0
------------------
5.100    1240
5.900    1107
4.500     943
7.800     824
5.700     819
5.800     753
7.300     612
6.700     597
4.300     532
6.800     517
7.200     495
5.400     448
3.500     439
4.600     417
5.200     416
6.300     412
7.500     364
4.200     353
6.600     352
4.100     320
4.700     289
7.100     260
7.600     251
3.900     245
6.200     227
6.900     223
5.500     216
7.700     205
8.700     182
7.400     179
6.500     167
6.100     153
8.200     143
6.400     143
4.400     135
5.600     107
5.300     101
7.900     101
4.800      72
4.900      58
3.700      53
3.400      50
8.600      42
9.600      42
8.300      41
8.500      39
9.200      34
3.300      30
3.800      26
8.100      25
0.200      21
1.300      19
9.900      16
9.400      14
9.100      12
8.400       7
9.800       5
1.200       4
8.900       4
1.500       4
0.800       3
9.500       3
0.100       2
3.600       2
8.800       2
0.400       1
9.700       1
Name: consump_city, dtyp

## `consump_country`

In [72]:
df_car.consump_country.isnull().sum()

3365

In [73]:
groupby_mode2("make_model","displacement","consump_country")

Number of NaN :  0
------------------
4.200    1562
3.700    1394
4.400    1325
3.900    1185
4.500    1105
3.800     945
4.700     849
4.900     755
4.100     740
3.100     707
3.300     702
4.600     632
3.600     540
3.500     525
4.300     525
5.300     370
3.400     353
4.800     341
5.700     242
5.400     239
5.100     235
3.200     180
5.600     154
5.200     118
6.300      52
5.500      36
5.800      30
7.700      20
6.600      19
6.400      15
6.500       4
7.300       4
7.100       3
5.900       2
6.100       2
7.800       2
6.700       2
7.600       1
0.300       1
8.600       1
2.900       1
6.900       1
Name: consump_country, dtype: int64


## `consump_comb`

In [74]:
df_car.consump_comb.isnull().sum()

3790

In [75]:
cons_comb = (df_car["consump_country"] + df_car["consump_city"])/2

In [76]:
cons_comb.isnull().sum()

0

In [77]:
df_car.consump_comb.fillna(cons_comb,inplace=True)

In [78]:
df_car["consump_comb"].value_counts(dropna=False)

4.500    1039
5.400     849
4.100     813
5.600     752
3.900     750
5.100     661
4.400     623
4.700     621
3.800     586
4.800     553
4.600     534
5.200     493
3.300     467
4.200     447
5.300     440
4.900     393
3.700     387
5.500     384
5.900     372
5.700     342
6.350     319
4.300     307
3.500     285
5.250     284
6.200     216
3.600     193
5.050     192
6.300     191
6.100     177
4.750     170
5.800     167
6.250     162
6.400     159
6.600     148
4.150     141
6.800     136
4.050     135
3.400     113
5.850     109
7.400      66
5.200      59
5.650      50
6.500      48
6.700      43
4.000      39
7.100      38
5.150      37
3.600      37
5.950      37
4.650      34
4.050      32
6.900      27
3.200      25
6.450      21
6.300      21
8.300      19
3.650      19
7.200      18
7.600      14
4.850      14
6.750      13
5.450      12
5.350       8
5.300       7
3.950       7
3.100       7
7.800       6
7.200       6
5.700       5
7.500       4
8.600       4
1.600 

In [79]:
df_car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15919 entries, 0 to 15918
Data columns (total 28 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   make_model           15919 non-null  object 
 1   body_type            15919 non-null  object 
 2   price                15919 non-null  int64  
 3   km                   15919 non-null  float64
 4   car_age              15919 non-null  float64
 5   prev_owner           15919 non-null  float64
 6   horse_power          15919 non-null  float64
 7   color                15919 non-null  object 
 8   paint_type           15919 non-null  object 
 9   gears                15919 non-null  float64
 10  safety_security      15919 non-null  object 
 11  type                 15919 non-null  object 
 12  upholstery           15919 non-null  object 
 13  nr_of_doors          15919 non-null  float64
 14  nr_of_seats          15919 non-null  float64
 15  gearing_type         15919 non-null 

## End of this phase

In [80]:
df_car.isnull().sum()/df_car.shape[0]*100

make_model            0.000
body_type             0.000
price                 0.000
km                    0.000
car_age               0.000
prev_owner            0.000
horse_power           0.000
color                 0.000
paint_type            0.000
gears                 0.000
safety_security       0.000
type                  0.000
upholstery            0.000
nr_of_doors           0.000
nr_of_seats           0.000
gearing_type          0.000
displacement          0.000
cylinders             0.000
weight_kg             0.000
drive_chain           0.000
fuel                  0.000
consump_comb          0.000
consump_city          0.000
consump_country       0.000
co2_emission          0.000
comfort_convenience   0.000
media                 0.000
extras                0.000
dtype: float64

In [82]:
df_car.to_csv("df_car_filled.csv", index=False)